# Baseline-решение

Основано на логистической регрессии и TF-IDF векторах для слов и для символов.

**Почему это имеет смысл:**
- TF-IDF для символов имеет смысл обучать для того, чтобы выучить смайлики или другие символы, которые не встречаются в обычной речи,
- TF-IDF для слов — так же, как и всегда,
- логистическая регрессия, видимо, хорошо работает как baseline-решение для классификации: по сути, мы учим не один классификатор сразу на 6 классов, а 6 классификаторов one-vs-all, по одному на каждый класс.

In [1]:
import pandas as pd
import numpy as np

## 1. Загружаем данные

In [2]:
class_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

train = pd.read_csv("./data/train.csv").fillna(" ")
test = pd.read_csv("./data/test.csv").fillna(" ")

train_text = train["comment_text"]
test_text = test["comment_text"]
all_text = pd.concat([train_text, test_text])

## 2. Препроцессинг данных

### 2.1 TF-IDF на словах

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents="unicode",
    analyzer="word",
    token_pattern=r"\w{1,}",
    stop_words="english",
    ngram_range=(1, 1),
    max_features=10000)

In [5]:
word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [6]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

### 2.2 Посимвольный TF-IDF

**NB:** размер n-грамм — от 2 до 6 (возможно, с этим можно поиграться).

In [7]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents="unicode",
    analyzer="char",
    stop_words="english",
    ngram_range=(2, 6),
    max_features=50000)

In [8]:
char_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(2, 6), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [9]:
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

### 2.3 Объединяем данные

По словам и по символам.

In [10]:
from scipy.sparse import hstack

In [11]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

## 3. Логистическая регрессия

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [13]:
scores = []
submission = pd.DataFrame.from_dict({"id": test["id"]})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver="sag")

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring="roc_auc"))
    scores.append(cv_score)
    print("CV score for class {} is {}".format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

CV score for class toxic is 0.9231798203736675
CV score for class severe_toxic is 0.9111330428057918
CV score for class obscene is 0.914619655278845
CV score for class threat is 0.9453560931421907
CV score for class insult is 0.8802011803030223
CV score for class identity_hate is 0.904822933001992


## 4. Результаты

### 4.1 Общий результат на кросс-валидации

In [14]:
print("Total CV score is {}".format(np.mean(scores)))

Total CV score is 0.9132187874842516


### 4.2 Сохраняем результаты

In [15]:
submission.to_csv("./submissions/baseline.csv", index=False)